In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

from preprocess import *


2023-12-05 17:22:23.110329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 17:22:23.270840: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-05 17:22:23.270866: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-05 17:22:23.313884: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 17:22:24.374016: W tensorflow/stream_executor/platform/de

In [5]:
df = pd.read_csv('/home/lewis/code/TrainDelays/TrainDelays/data/clean_data_final.csv')
df = df[df['PFPI_MINUTES'] != 0]


In [6]:
df.head()


,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,UNIT_CLASS_AFFECTED,INCIDENT_REASON,PERFORMANCE_EVENT_CODE,PFPI_MINUTES,Lat_OR,Lon_OR,...,ORIG_MINUTE_SIN,ORIG_MINUTE_COS,DEST_MONTH_SIN,DEST_MONTH_COS,DEST_DAY_SIN,DEST_DAY_COS,DEST_HOUR_SIN,DEST_HOUR_COS,DEST_MINUTE_SIN,DEST_MINUTE_COS
0,22215003,EK03,BH,Y,375.0,M,M,9.0,51.546500,-0.10408,...,-0.309017,0.951057,0.406737,0.913545,0.207912,0.978148,0.669131,-0.743145,-0.978148,2.079117e-01
1,22215003,EK03,BH,Y,375.0,M,M,6.0,51.546500,-0.10408,...,-0.951057,-0.309017,0.406737,0.913545,0.207912,0.978148,0.978148,-0.207912,-0.207912,-9.781476e-01
14,25234001,EK04,WD,Y,317.0,M,M,4.0,51.523880,-0.05982,...,-0.743145,0.669131,0.406737,0.913545,0.309017,0.951057,0.743145,0.669131,-0.309017,-9.510565e-01
15,25234001,EK04,WD,Y,317.0,M,M,5.0,51.701930,-0.02399,...,0.743145,-0.669131,0.406737,0.913545,0.309017,0.951057,0.809017,0.587785,0.309017,9.510565e-01
16,21235001,EK04,WD,Y,315.0,M,M,3.0,51.517991,-0.08140,...,-0.951057,0.309017,0.406737,0.913545,0.309017,0.951057,0.809017,0.587785,1.000000,2.832769e-16


In [7]:
pipe = preprocessing_pipe()


In [8]:
X = df.drop(columns=['PFPI_MINUTES'])
y = df['PFPI_MINUTES']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)


In [10]:
pipe.fit(X_train)


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  ['Lat_OR', 'Lon_OR',
                                                   'Lat_DES', 'Lon_DES']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['ENGLISH_DAY_TYPE',
                                                   'SERVICE_GROUP_CODE_AFFECTED',
                                                   'INCIDENT_REASON',
                                                   'UNIT_CLASS_AFFECTED',
                                                   'TRAIN_SERVICE_CODE_AFFECTED',
                                                   'PERFORMANCE_EVENT_CODE',
                                                   'APP_TIMETABLE_FLAG_AFF'])]))])

In [11]:
X_train_processed = pipe.transform(X_train)
X_test_processed = pipe.transform(X_test)


In [12]:
X_train_processed.shape


(313800, 69)

In [13]:
# Instantiate a sequential Neural Network
model = Sequential()


2023-12-05 17:24:36.771008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-05 17:24:36.771969: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-05 17:24:36.774780: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-LQCN2T3): /proc/driver/nvidia/version does not exist
2023-12-05 17:24:36.781324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# Instantiate a input layer
model.add(layers.Dense(128, input_dim=X_train_processed.shape[1], activation='relu', kernel_initializer='normal'))

# Adding Hideen layers
model.add(layers.Dense(256, activation='relu', kernel_initializer='normal'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(512, activation='relu', kernel_initializer='normal'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(512, activation='relu', kernel_initializer='normal'))
model.add(layers.Dropout(0.2))


# Final output layer
model.add(layers.Dense(1, activation='linear', kernel_initializer='normal'))


In [21]:
from tensorflow.keras import optimizers
adam_opt = optimizers.Adam(learning_rate=0.01)

# Compiling the neural network
model.compile(optimizer=adam_opt,
              loss='mse',
              metrics=['mae'])


In [22]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               8960      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dense_4 (Dense)             (None, 1)                 513       
                                                                 
 dense_5 (Dense)             (None, 128)               256       
                                                                 
 dense_6 (Dense)             (None, 256)               3

In [ ]:
es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train_processed, y_train, epochs=500,
          batch_size=64, validation_split=0.2, callbacks=[es])


2023-12-05 17:26:31.896150: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 69287040 exceeds 10% of free system memory.


Epoch 1/500
3923/3923 [==============================] - 60s 15ms/step - loss: 29.6567 - mae: 2.4151 - val_loss: 24.8835 - val_mae: 2.3384
Epoch 2/500
3923/3923 [==============================] - 64s 16ms/step - loss: 29.5962 - mae: 2.4060 - val_loss: 24.8849 - val_mae: 2.3560
Epoch 3/500
3923/3923 [==============================] - 62s 16ms/step - loss: 29.5563 - mae: 2.4031 - val_loss: 24.9248 - val_mae: 2.4123
Epoch 4/500
3923/3923 [==============================] - 57s 15ms/step - loss: 29.5533 - mae: 2.4064 - val_loss: 24.8832 - val_mae: 2.3423
Epoch 5/500
3923/3923 [==============================] - 74s 19ms/step - loss: 29.5545 - mae: 2.4044 - val_loss: 24.9046 - val_mae: 2.3873
Epoch 6/500
3923/3923 [==============================] - 68s 17ms/step - loss: 29.5542 - mae: 2.4057 - val_loss: 24.8878 - val_mae: 2.3639
Epoch 7/500
 389/3923 [=>............................] - ETA: 12:24 - loss: 34.0964 - mae: 2.4226